In [1]:
# Imports.
from g2p_client import G2PDatabase
from __future__ import division
import pandas as pd

In [2]:
# Constants
INDEX = 'associations'
HOST = 'localhost' # 'dms-dev.compbio.ohsu.edu'
DATABASE = G2PDatabase(HOST, INDEX)
GENIE_VARIANTS = '/Users/goecksj/Downloads/data_mutations_extended.txt'

In [3]:
# Load GENIE variants.
genie_variants = pd.read_csv(GENIE_VARIANTS, sep='\t', comment='#')

/Users/goecksj/miniconda2/envs/g2p-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,18,23,24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def in_g2p(row):
    '''
    Returns true if there are 1 or more results in G2P database.
    '''
    s = DATABASE.query_by_variant(row['Chromosome'], row['Start_Position'], row['End_Position'], row['Reference_Allele'], row['Tumor_Seq_Allele2'])
    return s.count() > 0

def summarize_results(row):
    '''
    Returns results summary for the variant.
    '''
    s = DATABASE.query_by_variant(row['Chromosome'], row['Start_Position'], row['End_Position'], row['Reference_Allele'], row['Tumor_Seq_Allele2'])        
    return DATABASE.summarize_results(s)


# genie_variants['in_g2p'] = genie_variants.apply(in_g2p, axis=1)
# in_g2p_vars = genie_variants[genie_variants['in_g2p'] == True]

results_summaries = genie_variants.apply(summarize_results, axis=1)
results_df = pd.DataFrame(list(results_summaries))
genie_variants = genie_variants.join(results_df)
genie_variants['total'] = genie_variants['total_A'] + genie_variants['total_B'] + genie_variants['total_C'] + genie_variants['total_D']

In [8]:
# Report statistics on variants found in database
in_g2p_vars = genie_variants[genie_variants['total'] > 0]
num_total_samples = len(genie_variants['Tumor_Sample_Barcode'].unique())
num_total_variants = len(genie_variants)
num_samples_in_g2p = len(in_g2p_vars['Tumor_Sample_Barcode'].unique())
num_variants_in_g2p = len(in_g2p_vars)

print 'Number of GENIE samples with variant(s) in G2P: %i/%i (%0.1f%%)' % \
    (num_samples_in_g2p, num_total_samples, num_samples_in_g2p/num_total_samples * 100)
print 'Number of GENIE variants in G2P: %i/%i (%0.1f%%)' % \
    (num_variants_in_g2p, num_total_variants, num_variants_in_g2p/num_total_variants * 100)

Number of GENIE samples with variant(s) in G2P: 7358/17163 (42.9%)
Number of GENIE variants in G2P: 8762/111476 (7.9%)


In [9]:
in_g2p_vars.groupby(['Hugo_Symbol', 'Chromosome', 'Start_Position', 'End_Position', 'Reference_Allele', 'Tumor_Seq_Allele2']).count()

Entrez_Gene_Id  \
Hugo_Symbol Chromosome Start_Position End_Position Reference_Allele Tumor_Seq_Allele2                   
ABL1        9          133738357      133738357    T                C                               1   
                       133747568      133747568    A                T                               1   
AKT1        14         105243048      105243048    G                T                               3   
                       105246551      105246551    C                T                             156   
ALK         2          29432664       29432664     C                T                               6   
                       29443613       29443613     C                T                               1   
                       29443631       29443631     G                T                               1   
                       29443695       29443695     G                T                               2   
ATM         11         108218044      108218044    A                C                               1   
BRAF        7          140453134      140453134    T                C                              29   
                       140453136      140453136    A                C                               1   
                                                                    T                             613   
                                      140453137    AC               TT                             15   
                       140453137      140453137    C                T                               9   
                       140453145      140453145    A                C                               5   
                                                                    T                               3   
                                      140453146    AG               GA                              2   
                       140453146      140453146    G                C                               2   
                       140453149      140453149    C                A                               2   
                       140453154      140453154    T                A                               2   
                                                                    C                              32   
                                                                    G                               2   
                       140453155      140453155    C                T                              27   
                       140477794      140477794    A                T                               1   
                       140481402      140481402    C                A                              10   
                                                                    G                              36   
                       140481411      140481411    C                A                              14   
CDK4        12         58145431       58145431     G                A                               3   
CTNNB1      3          41266124       41266124     A                G                              40   
                       41266136       41266136     T                C                              40   
...                                                                                               ...   
PTEN        10         89711899       89711899     C                T                              15   
                       89717672       89717672     C                T                              32   
RAC1        7          6426892        6426892      C                T                              13   
RAF1        3          12645699       12645699     G                A                              15   
RET         10         43617416       43617416     T                C                              16   
RUNX1       21         36252962       36252962     C                G                            